In [3]:
import numpy as np
from symtdd import ts,tn

In [4]:
a = np.random.random((2, 2))
b = np.random.random((2, 2))
ia = ts.Index('x', 0)
ib = ts.Index('x', 1)
ic = ts.Index('x', 2)
ta = ts.Tensor(a, [ia, ib])
tb = ts.Tensor(a, [ib, ic])

In [5]:
tna = tn.TensorNetwork([ta, tb])

In [6]:
set(ta.indices)

{x_0, x_1}

In [7]:
tc = tna.contract()

In [8]:
tc.data

array([[0.57619833, 0.46894472],
       [0.40317469, 0.42409264]])

In [9]:
tna.max_size

4

In [1]:
from symtdd.io.qiskit_import import cir_2_tn
from symtdd import DDType

In [2]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter

In [3]:
qa = QuantumRegister(2, 'qa')
qb = QuantumRegister(3, 'qb')
# va = Parameter('x0')
cir = QuantumCircuit(qa, qb)
# cir.rz(va, 3)
cir.cx(1, 4)
cir.cy(1, 4)
cir.ccx(1, 3, 4)
cir.h(2)

In [4]:
tna = cir_2_tn(cir, DDType.TDD)

In [ ]:
for tensor in tna.tensors:
    for index in tensor.indices:
        print(index)
    print("==")

In [6]:
tsa = tna.contract()

DEBUG:ts:(1,),(0,)->(0, 1)
DEBUG:ts:(1,),(0, 2)->(0, 1, 2)
DEBUG:ts:(3,),(0, 1, 2)->(0, 1, 2, 3)
DEBUG:ts:(3,),(0, 1, 4, 2)->(0, 1, 2, 4, 3)
DEBUG:ts:(0, 0, 5, 3),(0, 1, 2, 4, 5)->(1, 2, 3, 4)
DEBUG:ts:(0, 0, 4, 2),(1, 3, 4, 5)->(0, 1, 2, 3, 5)
DEBUG:ts:(0, 6, 3, 4, 2, 7),(0, 1, 2, 3, 5)->(6, 1, 4, 7, 5)
DEBUG:ts:(1, 2),(0, 1, 3, 4, 5)->(0, 2, 3, 4, 5)


In [7]:
tsa.data.shape

(2, 2, 2, 2, 2)

In [8]:
for index in tsa.indices:
    print(index)

y_0_0#0
y_3_0#0
y_1_0#0
y_2_0#0
y_4_0#0


In [11]:
import numpy as np
ordered_tsa_data = np.moveaxis(tsa.data, np.arange(5), [0, 3, 1, 2, 4])
ordered_tsa_data.flatten()

array([0.70710678+0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.70710678+0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.70710678j,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.70710678j,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ])

In [12]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
job = backend.run(cir)
result = job.result()
result

Result(backend_name='statevector_simulator', backend_version='0.12.0', qobj_id='', job_id='d0a7a062-b81d-4caf-911f-4441a274d910', success=True, results=[ExperimentResult(shots=1, success=True, meas_level=2, data=ExperimentResultData(statevector=Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j],
            dims=(2, 2, 2, 2, 2))), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, me

In [5]:
instra = cir.data[2]
instra

CircuitInstruction(operation=Instruction(name='ccx', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'qa'), 1), Qubit(QuantumRegister(3, 'qb'), 1), Qubit(QuantumRegister(3, 'qb'), 2)), clbits=())

In [18]:
type(instra.operation.base_gate)

qiskit.circuit.library.standard_gates.x.XGate

In [6]:
aa = instra.operation.to_matrix().reshape((2,)*6)
import numpy as np
aa.ndim
xposs = np.arange(aa.ndim//2)[::-1] * 2
yposs = xposs + 1
print(np.hstack((yposs, xposs)))
bb = np.moveaxis(aa, [i for i in range(aa.ndim)], np.hstack((yposs, xposs)))
bb[1, 1, 1, 1, :, :]

[5 3 1 4 2 0]


array([[0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j]])